# Building a Quantative Momentum Investing Strategy

**Disclaimer:** _Investing in the stock market involves risk and can lead to monetary loss. This material is purely for educational purposes and should not be taken as professional investment advice. Invest at your own discretion._

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an qual-weight portfolio of these 50 stocks.

Based on the course [Algorithmic Trading Using Python - Full Course](https://www.youtube.com/watch?v=xfzGZB4HhEE) by Nick McCullum from Youtube channel [freeCodeCamp.org](https://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ).

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API before processing. Make sure the '.csv' file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('./starter_files/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to gen one-year price returns for each stock in the universe. Here's how.
(Search the https://iexcloud.io/docs/api/ for "Key Stats")

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
data.status_code

200

In [5]:
data.json()

{'companyName': 'Apple Inc',
 'marketcap': 2313893672036,
 'week52high': 148.03,
 'week52low': 56.13,
 'week52change': 0.766970315282462,
 'sharesOutstanding': 17606177433,
 'float': 0,
 'avg10Volume': 143342753,
 'avg30Volume': 118034483,
 'day200MovingAvg': 123.29,
 'day50MovingAvg': 138.55,
 'employees': 0,
 'ttmEPS': 3.8,
 'ttmDividendRate': 0.819359603601632,
 'dividendYield': 0.006044309278747333,
 'nextDividendDate': '2021-02-03',
 'exDividendDate': '2020-11-02',
 'nextEarningsDate': '2021-01-17',
 'peRatio': 36.94196846594803,
 'beta': 1.1753797052207136,
 'maxChangePercent': 50.86636679118256,
 'year5ChangePercent': 5.12521229044863,
 'year2ChangePercent': 2.3911167783379574,
 'year1ChangePercent': 0.7772555134691216,
 'ytdChangePercent': 0.0111809130050726,
 'month6ChangePercent': 0.2759007026270877,
 'month3ChangePercent': 0.24210748084017206,
 'month1ChangePercent': 0.0110787753996106,
 'day30ChangePercent': 0.0114393348372828,
 'day5ChangePercent': -0.06216990721930879}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first projectof this course. Here is an example.

In [6]:
data = data.json()

In [7]:
data['year1ChangePercent']

0.7772555134691216

## Executing A Batch API & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100. 

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i: i+n]

In [9]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [10]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [11]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe= final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )

In [12]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,126.850,0.495615,N/A
1,AAL,16.956,-0.386006,N/A
2,AAP,157.230,0.160194,N/A
3,AAPL,139.693,0.780413,N/A
4,ABBV,106.900,0.341022,N/A
...,...,...,...,...
500,YUM,111.200,-0.00695947,N/A
501,ZBH,167.397,0.0511663,N/A
502,ZBRA,422.360,0.675051,N/A
503,ZION,47.380,0.0202437,N/A


## Removing Low-Momentum Stocks

This investment startegy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our Dataframe that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one year price return, and drop all stocks outside the top 50.

In [13]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)

In [14]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)

In [15]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,40.260,2.32474,N/A
1,179,FCX,28.760,1.61546,N/A
2,345,NVDA,554.350,1.28366,N/A
3,23,ALB,173.454,1.1619,N/A
4,387,PYPL,249.220,1.14282,N/A
5,24,ALGN,572.959,1.1278,N/A
6,388,QCOM,170.090,0.956545,N/A
7,410,SIVB,479.500,0.93377,N/A
8,490,WST,307.340,0.910307,N/A
9,31,AMD,92.320,0.899614,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [16]:
def portfolio_input():
    while True:
        portfolio_size = input('Enter the value of your portfolio:')

        try:
            val = float(portfolio_size)
            return val
            break
        except ValueError:
            print("That's not a number!\nPlease try again")

In [17]:
val = portfolio_input()

Enter the value of your portfolio:10000000


In [18]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] \
        = math.floor(position_size/final_dataframe.loc[i, 'Price'])

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [19]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,40.260,2.32474,4967
1,179,FCX,28.760,1.61546,6954
2,345,NVDA,554.350,1.28366,360
3,23,ALB,173.454,1.1619,1153
4,387,PYPL,249.220,1.14282,802
5,24,ALGN,572.959,1.1278,349
6,388,QCOM,170.090,0.956545,1175
7,410,SIVB,479.500,0.93377,417
8,490,WST,307.340,0.910307,650
9,31,AMD,92.320,0.899614,2166


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time.
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:
* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality-momentum`.

In [20]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

In [21]:
hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index=True
        )

In [22]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.340,N/A,0.477599,N/A,0.26799,N/A,0.186019,N/A,0.0214681,N/A,N/A
1,AAL,17.005,N/A,-0.385331,N/A,0.531395,N/A,0.505892,N/A,0.0704151,N/A,N/A
2,AAP,157.790,N/A,0.158264,N/A,0.00874218,N/A,0.0271051,N/A,-0.0425364,N/A,N/A
3,AAPL,139.481,N/A,0.782991,N/A,0.271779,N/A,0.238531,N/A,0.0113056,N/A,N/A
4,ABBV,106.100,N/A,0.332095,N/A,0.109236,N/A,0.220781,N/A,-0.0353639,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.189,N/A,-0.00703116,N/A,0.144031,N/A,0.112511,N/A,-0.0511639,N/A,N/A
501,ZBH,161.420,N/A,0.0501235,N/A,0.149715,N/A,0.168242,N/A,-0.0012641,N/A,N/A
502,ZBRA,417.100,N/A,0.679763,N/A,0.422518,N/A,0.418945,N/A,0.0349154,N/A,N/A
503,ZION,46.590,N/A,0.0202006,N/A,0.404495,N/A,0.415924,N/A,0.0293901,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:
* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [23]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

There is a problem with the data - missing values. Fix from: https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float

In [24]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] is None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

In [25]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        percentile_col = f'{time_period} Return Percentile'
        change_col = f'{time_period} Price Return'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],
                                                       hqm_dataframe.loc[row, change_col])/100


In [26]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.340,N/A,0.477599,0.871287,0.26799,0.649505,0.186019,0.552475,0.0214681,0.643564,N/A
1,AAL,17.005,N/A,-0.385331,0.0217822,0.531395,0.90297,0.505892,0.932673,0.0704151,0.847525,N/A
2,AAP,157.790,N/A,0.158264,0.580198,0.00874218,0.176238,0.0271051,0.154455,-0.0425364,0.285149,N/A
3,AAPL,139.481,N/A,0.782991,0.958416,0.271779,0.659406,0.238531,0.645545,0.0113056,0.576238,N/A
4,ABBV,106.100,N/A,0.332095,0.784158,0.109236,0.364356,0.220781,0.621782,-0.0353639,0.342574,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.189,N/A,-0.00703116,0.332673,0.144031,0.423762,0.112511,0.364356,-0.0511639,0.217822,N/A
501,ZBH,161.420,N/A,0.0501235,0.427723,0.149715,0.431683,0.168242,0.50495,-0.0012641,0.50297,N/A
502,ZBRA,417.100,N/A,0.679763,0.940594,0.422518,0.827723,0.418945,0.865347,0.0349154,0.720792,N/A
503,ZION,46.590,N/A,0.0202006,0.382178,0.404495,0.811881,0.415924,0.861386,0.0293901,0.677228,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score` , which is the high quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that will be calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from the Python's built-in `statistics` module.

In [27]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.340,N/A,0.477599,0.871287,0.26799,0.649505,0.186019,0.552475,0.0214681,0.643564,0.679208
1,AAL,17.005,N/A,-0.385331,0.0217822,0.531395,0.90297,0.505892,0.932673,0.0704151,0.847525,0.676238
2,AAP,157.790,N/A,0.158264,0.580198,0.00874218,0.176238,0.0271051,0.154455,-0.0425364,0.285149,0.29901
3,AAPL,139.481,N/A,0.782991,0.958416,0.271779,0.659406,0.238531,0.645545,0.0113056,0.576238,0.709901
4,ABBV,106.100,N/A,0.332095,0.784158,0.109236,0.364356,0.220781,0.621782,-0.0353639,0.342574,0.528218
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.189,N/A,-0.00703116,0.332673,0.144031,0.423762,0.112511,0.364356,-0.0511639,0.217822,0.334653
501,ZBH,161.420,N/A,0.0501235,0.427723,0.149715,0.431683,0.168242,0.50495,-0.0012641,0.50297,0.466832
502,ZBRA,417.100,N/A,0.679763,0.940594,0.422518,0.827723,0.418945,0.865347,0.0349154,0.720792,0.838614
503,ZION,46.590,N/A,0.0202006,0.382178,0.404495,0.811881,0.415924,0.861386,0.0293901,0.677228,0.683168


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM score` column and dropping all but the top 50 entries 

In [28]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)

In [29]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,173.856,N/A,1.13467,0.990099,1.07695,0.990099,0.800971,0.978218,0.133774,0.942574,0.975248
1,SIVB,474.230,N/A,0.900979,0.982178,1.05809,0.986139,0.582021,0.960396,0.185421,0.972277,0.975248
2,VIAC,52.500,N/A,0.615404,0.912871,1.06024,0.988119,0.859372,0.982178,0.421255,1,0.970792
3,FCX,29.512,N/A,1.57103,0.99802,1.22493,0.994059,0.631253,0.968317,0.0891655,0.881188,0.960396
4,AMAT,107.010,N/A,0.777823,0.956436,0.597652,0.930693,0.744247,0.976238,0.174231,0.968317,0.957921
5,DISCA,40.940,N/A,0.453741,0.853465,1.03327,0.984158,1.11222,0.994059,0.405179,0.99802,0.957426
6,GM,53.639,N/A,0.585517,0.910891,1.09101,0.992079,0.501778,0.928713,0.24344,0.992079,0.955941
7,IVZ,22.040,N/A,0.317518,0.770297,1.23689,0.99604,0.658331,0.972277,0.236162,0.988119,0.931683
8,KLAC,311.000,N/A,0.798166,0.960396,0.478728,0.877228,0.508341,0.934653,0.133723,0.940594,0.928218
9,URI,254.690,N/A,0.881769,0.974257,0.660427,0.946535,0.437336,0.887129,0.0959196,0.90495,0.928218


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [30]:
val = portfolio_input()

Enter the value of your portfolio:1000000


In [31]:
position_size = val/len(hqm_dataframe.index)

for i in range(0, len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] \
        = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [32]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,173.856,115,1.13467,0.990099,1.07695,0.990099,0.800971,0.978218,0.133774,0.942574,0.975248
1,SIVB,474.230,42,0.900979,0.982178,1.05809,0.986139,0.582021,0.960396,0.185421,0.972277,0.975248
2,VIAC,52.500,380,0.615404,0.912871,1.06024,0.988119,0.859372,0.982178,0.421255,1,0.970792
3,FCX,29.512,677,1.57103,0.99802,1.22493,0.994059,0.631253,0.968317,0.0891655,0.881188,0.960396
4,AMAT,107.010,186,0.777823,0.956436,0.597652,0.930693,0.744247,0.976238,0.174231,0.968317,0.957921
5,DISCA,40.940,488,0.453741,0.853465,1.03327,0.984158,1.11222,0.994059,0.405179,0.99802,0.957426
6,GM,53.639,372,0.585517,0.910891,1.09101,0.992079,0.501778,0.928713,0.24344,0.992079,0.955941
7,IVZ,22.040,907,0.317518,0.770297,1.23689,0.99604,0.658331,0.972277,0.236162,0.988119,0.931683
8,KLAC,311.000,64,0.798166,0.960396,0.478728,0.877228,0.508341,0.934653,0.133723,0.940594,0.928218
9,URI,254.690,78,0.881769,0.974257,0.660427,0.946535,0.437336,0.887129,0.0959196,0.90495,0.928218


## Formatting Our Excel Output

We will be using the XlsxWriter for Pyton to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [33]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, 'Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts and also symbols like % and $. We'll need four main formulas for our Excel document:

* String format for tickers
* `$XX.XX` format for stock prices
* `$XX.XXX` format for market capitalization
* Integer format for the numer of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code before proceeding.

In [34]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00', 
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [35]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', percent_format],
    'E': ['One-Year Return Percentile', percent_format],
    'F': ['Six-Month Price Return', percent_format],
    'G': ['Six-Month Return Percentile', percent_format],
    'H': ['Three-Month Price Return', percent_format],
    'I': ['Three-Month Return Percentile', percent_format],
    'J': ['One-Month Price Return', percent_format],
    'K': ['One-Month Return Percentile', percent_format],
    'L': ['HQM Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].write(f'{column}1',
                                             column_formats[column][0],
                                             column_formats[column][1])
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel file is very easy:

THIS IS NOT AN INVESTMENT ADVICE!

In [36]:
writer.save()